In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.ikea.com/us/en/p/vimle-sofa-dalstorp-multicolor-s59298777/'
response = requests.get(url)
response

<Response [200]>

In [6]:
soup = BeautifulSoup(response.text, "html.parser")
brand = soup.findAll("span", {"class":"product-pip__name"})[0].contents[0]
price = soup.findAll("span", {"class":"product-pip__price__value"})[0].contents[0]
dimensions = soup.findAll("dd", {"class":"product-pip__definition-list-item"})

#copying all dimensions for now, we'll figure out on what values are important in dimensions
d_list = []
for i in range(len(dimensions)):
    d_list.append(dimensions[i].contents[0])
    
imgLink = soup.findAll("img", {"class":""})[0]['src']

brand, price, d_list, imgLink

#Width
#soup.findAll("dd", {"class":"product-pip__definition-list-item"})[1].contents[0]

# pip__name - Name of company
# normal-font range__text-rtl - product Description
# pip-price-component - Price obviously
# pip__definition-list-item (Every second) - Width, Depth, Height

('VIMLE',
 '$599.00',
 ['31 1/2 "',
  '94 7/8 "',
  '38 5/8 "',
  '1 5/8 "',
  '5 7/8 "',
  '25 5/8 "',
  '83 1/8 "',
  '21 5/8 "',
  '17 3/4 "'],
 'https://www.ikea.com/PIAimages/0787230_PE763193_S5.JPG?f=s')